---
description: Core Classes for hwpapi
output-file: core.html
title: core

---

In [1]:
#| hide
%load_ext autoreload
%autoreload 2

In [2]:
#| default_exp core

In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
#| hide
#| export
from contextlib import contextmanager
from pathlib import Path
import numbers
import hwpapi.constants as const
from fastcore.basics import patch
import sys
import warnings
from hwpapi.logging import get_logger

In [5]:
#| hide
#| export
from hwpapi.actions import _Action, _Actions
from hwpapi.parametersets import ParaShape
import hwpapi.parametersets as parametersets
from hwpapi.classes import MoveAccessor, CellAccessor, TableAccessor, PageAccessor
from hwpapi.functions import (
    check_dll,
    get_hwp_objects,
    dispatch,
    get_absolute_path,
    get_rgb_tuple,
    set_pset,
)

In [6]:
#| export
class Engine:
    """
    한컴오피스 Hwp 객체를 캡슐화하는 Engine 클래스입니다.

    이 클래스는 한컴오피스 Hwp 애플리케이션과 상호작용하는 인터페이스를 제공하며,
    Hwp 환경 내에서 작업과 동작을 용이하게 합니다.

    매개변수
    ----------
    hwp_object : object, optional
        Engine에 의해 캡슐화될 Hwp 객체. 제공되지 않은 경우, 
        "HWPFrame.HwpObject"를 사용하여 새로운 Hwp 객체를 생성합니다.

    속성
    ----------
    impl : object
        Hwp 객체의 구현체.

    메서드
    -------
    name()
        Hwp 객체의 이름(CLSID)을 반환합니다.

    사용 예시
    --------
    >>> engine = Engine()
    >>> print(engine.name)
    """

    def __init__(self, hwp_object=None):
        """
        한컴 Hwp 객체로 Engine을 초기화합니다.

        `hwp_object`가 제공되지 않으면 새로운 Hwp 객체가 생성됩니다. 생성에 실패하면 `impl`이 None으로 설정됩니다.

        매개변수
        ----------
        hwp_object : object, optional
            Engine에 의해 캡슐화될 Hwp 객체. 기본값은 "HWPFrame.HwpObject"입니다.
        """
        self.logger = get_logger('core')
        try:
            if not hwp_object:
                hwp_object = "HWPFrame.HwpObject"
            self.logger.debug(f"Initializing Engine with hwp_object: {hwp_object}")
            self.impl = dispatch(hwp_object)
            self.logger.info(f"Engine initialized successfully with CLSID: {self.impl.CLSID if self.impl else 'None'}")
        except Exception as e:
            self.logger.error(f"Failed to initialize Hwp object: {e}")
            self.impl = None

    @property
    def name(self):
        """
        Hwp 객체의 이름(CLSID)을 반환합니다.

        반환값
        -------
        str
            Hwp 객체의 CLSID. 객체가 초기화되지 않은 경우 None을 반환합니다.
        """
        return self.impl.CLSID if self.impl else None

    def __repr__(self):
        """
        Engine 객체의 문자열 표현을 반환합니다.

        반환값
        -------
        str
            Engine의 문자열 표현. 엔진이 초기화되지 않은 경우 'Uninitialized'로 표시됩니다.
        """
        engine_name = self.name if self.name else "Uninitialized"
        return f"<Engine {engine_name}>"


In [7]:
#| export
class Engines:
    """
    여러 Engine 인스턴스를 관리하는 컬렉션 매니저입니다.

    이 클래스는 여러 Engine 인스턴스를 관리하며, 이들에 접근하고 반복하는 메서드를 제공합니다.
    여러 한컴오피스 Hwp 객체를 처리하는 데 유용합니다.

    매개변수
    ----------
    dll_path : str, optional
        초기화에 필요한 경우 DLL 파일의 경로.

    속성
    ----------
    active : Engine or None
        현재 활성화된 Engine 인스턴스.
    engines : list of Engine
        이 컬렉션에서 관리하는 Engine 인스턴스 목록.

    메서드
    -------
    add(engine)
        컬렉션에 새로운 Engine 인스턴스를 추가합니다.
    count()
        컬렉션의 Engine 인스턴스 수를 반환합니다.

    사용 예시
    --------
    >>> engines = Engines()
    >>> engines.add(Engine())
    >>> print(engines.count)
    >>> for engine in engines:
    ...     print(engine)

    주의사항
    -----
    `Engines` 클래스는 `get_hwp_objects()` 함수에서 검색된 각 객체에 대해 Engine 인스턴스를 생성하여 초기화됩니다. 
    `dll_path`가 제공된 경우 필요한 DLL을 확인합니다.
    """

    def __init__(self, dll_path=None):
        """
        사용 가능한 Hwp 객체로 Engines 컬렉션을 초기화합니다.

        매개변수
        ----------
        dll_path : str, optional
            초기화에 필요한 경우 DLL 파일의 경로.
        """
        self.active = None
        self.engines = [Engine(hwp_object) for hwp_object in get_hwp_objects()]
        check_dll(dll_path)

    def add(self, engine):
        """
        컬렉션에 새로운 Engine 인스턴스를 추가합니다.

        매개변수
        ----------
        engine : Engine
            컬렉션에 추가할 Engine 인스턴스.
        """
        self.engines.append(engine)

    @property
    def count(self):
        """
        컬렉션의 Engine 인스턴스 수를 반환합니다.

        반환값
        -------
        int
            Engine 인스턴스의 수.
        """
        return len(self)

    def __call__(self, ind):
        """
        인덱스를 기준으로 컬렉션에서 Engine 인스턴스를 반환합니다.

        매개변수
        ----------
        ind : int
            검색할 Engine 인스턴스의 인덱스.

        반환값
        -------
        Engine
            지정된 인덱스의 Engine 인스턴스.
        """
        if isinstance(ind, numbers.Number):
            return self.engines[ind]

    def __getitem__(self, ind):
        """
        인덱스를 기준으로 컬렉션에서 Engine 인스턴스를 반환합니다.

        매개변수
        ----------
        ind : int
            검색할 Engine 인스턴스의 인덱스.

        반환값
        -------
        Engine
            지정된 인덱스의 Engine 인스턴스.
        """
        if isinstance(ind, numbers.Number):
            return self.engines[ind]

    def __len__(self):
        """
        컬렉션의 Engine 인스턴스 수를 반환합니다.

        반환값
        -------
        int
            Engine 인스턴스의 수.
        """
        return len(self.engines)

    def __iter__(self):
        """
        컬렉션의 각 Engine 인스턴스를 반환합니다.

        반환값
        ------
        Engine
            컬렉션의 각 Engine 인스턴스.
        """
        for engine in self.engines:
            yield engine

    def __repr__(self):
        """
        Engines 컬렉션의 문자열 표현을 반환합니다.

        반환값
        -------
        str
            Engines 컬렉션의 문자열 표현.
        """
        return f"<HWP Engines activated: {len(self.engines)}>"


In [8]:
#| export

class Apps:
    """
    모든 `app <App>` 객체의 컬렉션입니다.

    속성
    ----------
    _apps : list
        `App` 인스턴스를 포함하는 리스트.

    메서드
    -------
    add(**kwargs)
        새로운 App을 생성하고 컬렉션에 추가합니다.
    count()
        컬렉션의 앱 수를 반환합니다.
    cleanup()
        [메서드 설명 필요]
    """

    def __init__(self):
        """
        `Engines`의 각 `engine`에 대해 `App` 인스턴스를 생성하여 `Apps` 컬렉션을 초기화합니다.
        """
        self._apps = [App(engine=engine) for engine in Engines()]

    def add(self, **kwargs):
        """
        새로운 App을 생성합니다. 새로운 App이 활성화됩니다.

        반환값
        -------
        App
            새로 생성된 App 객체.
        """
        app = App(engine=Engine())
        self._apps.append(app)
        return app  

    def __call__(self, i):
        """
        `Apps` 인스턴스를 함수처럼 호출할 수 있게 하여, 지정된 인덱스의 앱을 반환합니다.

        매개변수
        ----------
        i : int
            검색할 앱의 인덱스.

        반환값
        -------
        App
            지정된 인덱스의 앱.
        """
        return self[i]

    def __repr__(self):
        """
        `Apps` 인스턴스의 문자열 표현을 반환합니다.

        반환값
        -------
        str
            `Apps` 인스턴스의 문자열 표현.
        """
        return "{}({})".format(
            getattr(self.__class__, "_name", self.__class__.__name__), repr(list(self))
        )

    def __getitem__(self, item):
        """
        지정된 인덱스의 앱을 반환합니다.

        매개변수
        ----------
        item : int
            앱의 인덱스.

        반환값
        -------
        App
            지정된 인덱스의 앱.
        """
        return self._apps[item]

    def __len__(self):
        """
        컬렉션의 앱 수를 반환합니다.

        반환값
        -------
        int
            앱의 수.
        """
        return len(self._apps)

    @property
    def count(self):
        """
        컬렉션의 앱 수를 반환합니다.

        .. versionadded:: 0.9.0

        반환값
        -------
        int
            앱의 수.
        """
        return len(self)

    def cleanup(self):
        """
        [메서드 설명 필요]

        [메서드가 수행하는 작업과 부작용에 대한 설명]
        """
        

    def __iter__(self):
        """
        앱 컬렉션에 대한 반복을 허용합니다.

        반환값
        ------
        App
            컬렉션의 다음 앱.
        """
        for app in self._apps:
            yield app


In [9]:
# | export
class App:
    """
    `App` 클래스는 한컴오피스의 한/글 프로그램과 상호작용하기 위한 인터페이스를 제공합니다.

    이 클래스는 한/글 프로그램의 COM 객체와의 연동을 관리하고, 해당 객체에 대한 작업을 수행하는 메서드들을 포함합니다.

    Attributes
    ----------
    engine : Engine, optional
        사용할 엔진 객체입니다.
    is_visible : bool, optional
        한/글 프로그램의 가시성을 설정합니다.
    dll_path : str, optional
        사용할 DLL 파일의 경로입니다.

    Methods
    -------
    __init__(engine=None, is_visible=True, dll_path=None)
        `App` 클래스의 인스턴스를 초기화합니다.
    api()
        현재 사용 중인 엔진의 구현체를 반환합니다.
    __str__()
        `App` 인스턴스의 문자열 표현을 반환합니다.
    """

    def __init__(
        self,
        new_app=False,
        is_visible=True,
        dll_path=None,
        engine=None,
    ):
        """
        `App` 클래스의 인스턴스를 초기화합니다.

        Parameters
        ----------
        new_app: bool, optional
            별도 앱을 열지를 결정합니다.
        is_visible : bool, optional
            한/글 프로그램의 가시성 설정. 기본값은 True입니다.
        dll_path : str, optional
            사용할 DLL 파일의 경로. 기본값은 None입니다.
        engine : Engine, optional
            한/글 프로그램과의 상호작용을 위한 엔진. 기본값이 없을 경우, 새로운 엔진 인스턴스가 생성됩니다.
        Notes
        -----
        engine이 제공되지 않은 경우, `Engines`를 통해 엔진을 생성하거나 선택합니다.
        """
        self.logger = get_logger('core')
        self.logger.debug(f"Initializing App with new_app={new_app}, is_visible={is_visible}, dll_path={dll_path}")
        
        self._load(new_app=new_app, dll_path=dll_path, engine=engine)
        self.actions = _Actions(self)
        self.parameters = self.api.HParameterSet
        self.set_visible(is_visible)
        self.logger.info(f"App window visibility set to: {is_visible}")
        
        self.move = MoveAccessor(self)
        self.cell = CellAccessor(self)
        self.table = TableAccessor(self)
        self.page = PageAccessor(self)
        self.logger.info("App initialized successfully with all accessors")
        
    def _load(self, new_app=False, engine=None, dll_path=None):
        self.logger.debug(f"Loading App with new_app={new_app}, engine={engine}, dll_path={dll_path}")
        
        if new_app:
            engine = Engine()
            self.logger.debug("Created new engine for new_app")
        if not engine:
            engines = Engines()
            engine = engines[-1] if len(engines) > 0 else Engine()
            self.logger.debug(f"Selected engine from engines collection: {len(engines.engines) if engines else 0} available")
        
        self.engine = engine
        self.logger.info(f"Engine loaded successfully: {engine.name if engine and engine.impl else 'No engine'}")
        
        # Try multiple locations for DLL file
        if dll_path is None:
            self.logger.debug("Searching for DLL file in multiple locations")
            # First try the bundled DLL path
            if hasattr(sys, '_MEIPASS'):  # Check if running as PyInstaller bundle
                bundled_dll = Path(sys._MEIPASS) / 'FilePathCheckerModuleExample.dll'
                if bundled_dll.exists():
                    dll_path = bundled_dll
                    self.logger.debug(f"Found DLL in PyInstaller bundle: {dll_path}")
            
            # Then try current directory
            if dll_path is None:
                local_dll = Path.cwd() / 'FilePathCheckerModuleExample.dll'
                if local_dll.exists():
                    dll_path = local_dll
                    self.logger.debug(f"Found DLL in current directory: {dll_path}")
                
            # Finally try package directory
            if dll_path is None:
                package_dir = Path(__file__).parent
                package_dll = package_dir / 'FilePathCheckerModuleExample.dll'
                if package_dll.exists():
                    dll_path = package_dll
                    self.logger.debug(f"Found DLL in package directory: {dll_path}")
        
        # Register DLL if found
        if dll_path is not None:
            self.logger.info(f"Registering DLL: {dll_path}")
            check_dll(dll_path)
        else:
            self.logger.warning("No DLL file found - some functionality may be limited")
        
        try:
            self.api.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")
            self.logger.debug("Successfully registered FilePathCheckDLL module")
        except Exception as e:
            self.logger.warning(f"Failed to register FilePathCheckDLL module: {e}")
            warnings.warn(f"Failed to register FilePathCheckDLL module: {e}")
        

    @property
    def api(self):
        """
        현재 사용 중인 엔진의 구현체를 반환합니다.

        Returns
        -------
        object
            엔진의 구현체.
        """
        return self.engine.impl

    def __str__(self):
        """
        `App` 인스턴스의 문자열 표현을 반환합니다.

        Returns
        -------
        str
            `App` 인스턴스를 설명하는 문자열.
        """
        return f"<Hwp App: {self.get_filepath()}>"

    __repr__ = __str__

    # add attribution for patched functions for autocompletion
    close = 'patch' 
    create_action = 'patch'
    create_parameterset = 'patch'
    engine = 'patch'
    find_text = 'patch'
    charshape = "patch"
    get_charshape = 'patch'
    set_charshape = 'patch'
    get_filepath = 'patch'
    get_font_list = 'patch'
    get_hwnd = 'patch'
    get_parashape = 'patch'
    get_selected_text = 'patch'
    get_text = 'patch'
    insert_file = 'patch'
    insert_picture = 'patch'
    insert_text = 'patch'
    open = 'patch'
    quit = 'patch'
    reload = 'patch'
    replace_all = 'patch'
    save = 'patch'
    save_block = 'patch'
    scan = 'patch'
    select_text = 'patch'
    set_cell_border = 'patch'
    set_cell_color = 'patch'
    set_charshape = 'patch'
    set_parashape = 'patch'
    set_visible = 'patch'
    setup_page = 'patch'


In [10]:
#| export
@patch
def set_visible(app: App, is_visible=True, window_i=0):
    """
    한컴오피스 Hwp 프로그램 창의 가시성을 설정합니다.

    이 함수는 `is_visible` 매개변수에 따라 한컴오피스 Hwp 프로그램의 창을 표시하거나 숨기는 데 사용됩니다.
    `window_i` 매개변수는 표시하거나 숨길 창의 인덱스를 지정합니다.

    매개변수
    ----------
    app : App
        한컴오피스 Hwp 프로그램과 연결된 `App` 인스턴스.
    is_visible : bool, optional
        프로그램 창의 가시성을 나타내는 불린 값. 
        True이면 창이 표시되고, False이면 숨겨집니다. 기본값은 True입니다.
    window_i : int, optional
        수정할 창의 인덱스. 기본값은 0입니다.

    사용 예시
    --------
    >>> app = App()
    >>> set_visible(app, is_visible=True, window_i=0)
    >>> set_visible(app, is_visible=False, window_i=1)
    """
    logger = get_logger('core')
    logger.debug(f"Calling set_visible")
    app.api.XHwpWindows.Item(window_i).Visible = is_visible

In [11]:
#| export


@patch
def create_parameterset(app: App, key:str):
    logger = get_logger('core')
    logger.debug(f"Calling create_parameterset")
    return getattr(app.api.HParameterSet, f"H{key}")

In [12]:
#| hide

from hwpapi.core import App

app = App()

app.set_visible(True)

2025-12-08 13:22:21 - hwpapi.functions.get_hwp_objects - INFO - Found 1 running HWP objects
2025-12-08 13:22:21 - hwpapi.functions.dispatch - INFO - Successfully dispatched: <PyIDispatch at 0x000001F7F6D89140 with obj at 0x000001F7F8122F78>
2025-12-08 13:22:21 - hwpapi.core - INFO - Engine initialized successfully with CLSID: {5E6A8276-CF1C-42B8-BCED-319548B02AF6}
2025-12-08 13:22:21 - hwpapi.core - INFO - Engine loaded successfully: {5E6A8276-CF1C-42B8-BCED-319548B02AF6}
2025-12-08 13:22:21 - hwpapi.core - INFO - Registering DLL: C:\Users\freed\Documents\python_projects\007_hwpapi\hwpapi\FilePathCheckerModuleExample.dll
2025-12-08 13:22:21 - hwpapi.core - INFO - App window visibility set to: True
2025-12-08 13:22:21 - hwpapi.core - INFO - App initialized successfully with all accessors


In [13]:
#| hide
app.create_parameterset("CharShape")

<win32com.gen_py.HwpObject 1.0 Type Library.HCharShape instance at 0x2164561261488>

In [14]:
#| export

@patch
def reload(app: App, new_app=False, dll_path=None):
    """
    새로운 HWPFrame.HwpObject로 `App` 인스턴스를 다시 로드하고 가시성과 DLL 경로를 재설정합니다.

    이 함수는 `app` 객체의 API를 새로운 HWPFrame.HwpObject 인스턴스로 다시 초기화하도록 설계되었습니다.
    또한 제공된 경우 지정된 DLL 파일을 확인하고 등록합니다.

    매개변수
    ----------
    app : App
        다시 로드할 `App` 인스턴스.
    dll_path : str, optional
        확인하고 등록할 DLL 파일의 경로. None인 경우 DLL 등록이 수행되지 않습니다.

    주의사항
    -----
    `reload` 함수는 HWPFrame.HwpObject의 상태를 재설정해야 하거나
    사용 중인 DLL을 변경할 때 유용합니다.

    사용 예시
    --------
    >>> app = App()
    >>> reload(app, dll_path="path/to/dll")
    """
    logger = get_logger('core')
    logger.debug(f"Calling reload")
    app._load(new_app=new_app, dll_path=dll_path)

In [15]:
#| hide

# test reload

app = App()
app.reload()

2025-12-08 13:22:21 - hwpapi.functions.get_hwp_objects - INFO - Found 1 running HWP objects
2025-12-08 13:22:21 - hwpapi.functions.dispatch - INFO - Successfully dispatched: <PyIDispatch at 0x000001F7F6D88F60 with obj at 0x000001F7F8122F78>
2025-12-08 13:22:21 - hwpapi.core - INFO - Engine initialized successfully with CLSID: {5E6A8276-CF1C-42B8-BCED-319548B02AF6}
2025-12-08 13:22:21 - hwpapi.core - INFO - Engine loaded successfully: {5E6A8276-CF1C-42B8-BCED-319548B02AF6}
2025-12-08 13:22:21 - hwpapi.core - INFO - Registering DLL: C:\Users\freed\Documents\python_projects\007_hwpapi\hwpapi\FilePathCheckerModuleExample.dll
2025-12-08 13:22:21 - hwpapi.core - INFO - App window visibility set to: True
2025-12-08 13:22:21 - hwpapi.core - INFO - App initialized successfully with all accessors
2025-12-08 13:22:21 - hwpapi.functions.get_hwp_objects - INFO - Found 1 running HWP objects
2025-12-08 13:22:21 - hwpapi.functions.dispatch - INFO - Successfully dispatched: <PyIDispatch at 0x000001F7F6

In [16]:
#| export
@patch
def get_filepath(app: App):
    """
    현재 열려있는 한컴오피스 Hwp 문서의 파일 경로를 검색합니다.

    이 함수는 `App` 인스턴스와 연결된 한컴오피스 Hwp 프로그램의 활성 문서에 접근하여
    전체 파일 경로를 반환합니다.

    매개변수
    ----------
    app : App
        한컴오피스 Hwp 프로그램과 연결된 `App` 인스턴스.

    반환값
    -------
    str
        현재 활성화된 한컴오피스 Hwp 문서의 전체 파일 경로.

    사용 예시
    --------
    >>> app = App()
    >>> filepath = get_filepath(app)
    >>> print(filepath)
    """
    logger = get_logger('core')
    logger.debug(f"Calling get_filepath")
    doc = app.api.XHwpDocuments.Active_XHwpDocument
    return doc.FullName

In [17]:
#| hide

app.get_filepath()

''

In [18]:
#| export
@patch
def create_action(app: App, action_key: str):
    """
    `_Action` 클래스의 인스턴스를 생성하고 반환합니다.

    이 함수는 주어진 `app`과 `action_key`와 연결된 새로운 `_Action` 인스턴스를 생성합니다. 
    `action_key`는 애플리케이션에 대해 생성할 특정 액션의 유형을 지정합니다.

    매개변수
    ----------
    app : App
        액션이 생성될 애플리케이션 객체.
    action_key : str
        생성할 특정 액션을 나타내는 키.

    반환값
    -------
    _Action
        제공된 애플리케이션 객체와 액션 키로 초기화된 `_Action` 클래스의 인스턴스.

    사용 예시
    --------
    >>> app = App()
    >>> action = create_action(app, 'some_action_key')
    >>> print(action)
    """
    logger = get_logger('core')
    logger.debug(f"Calling create_action")
    return _Action(app, action_key)

In [19]:
#| hide
app = App()
app.create_action("FindDlg")

2025-12-08 13:22:22 - hwpapi.functions.get_hwp_objects - INFO - Found 1 running HWP objects
2025-12-08 13:22:22 - hwpapi.functions.dispatch - INFO - Successfully dispatched: <PyIDispatch at 0x000001F7F6D89080 with obj at 0x000001F7F8122F78>
2025-12-08 13:22:22 - hwpapi.core - INFO - Engine initialized successfully with CLSID: {5E6A8276-CF1C-42B8-BCED-319548B02AF6}
2025-12-08 13:22:22 - hwpapi.core - INFO - Engine loaded successfully: {5E6A8276-CF1C-42B8-BCED-319548B02AF6}
2025-12-08 13:22:22 - hwpapi.core - INFO - Registering DLL: C:\Users\freed\Documents\python_projects\007_hwpapi\hwpapi\FilePathCheckerModuleExample.dll
2025-12-08 13:22:22 - hwpapi.core - INFO - App window visibility set to: True
2025-12-08 13:22:22 - hwpapi.core - INFO - App initialized successfully with all accessors


<Action FindDlg: 찾기>

In [20]:
#| export
@patch
def open(app: App, path: str):
    """
    제공된 파일 경로를 사용하여 한컴오피스 Hwp 프로그램에서 파일을 엽니다.

    이 함수는 먼저 제공된 파일 경로를 `get_absolute_path` 함수를 사용하여 절대 경로로 변환합니다.
    그런 다음 `api.Open` 메서드를 사용하여 한컴오피스 Hwp 프로그램에서 파일을 열고 열린 파일의 절대 경로를 반환합니다.

    매개변수
    ----------
    app : App
        한컴오피스 Hwp 프로그램과 연결된 `App` 인스턴스.
    path : str
        열릴 문서의 파일 경로.

    반환값
    -------
    str
        열린 파일의 절대 경로.

    사용 예시
    --------
    >>> app = App()
    >>> opened_file_path = open(app, 'path/to/document.hwp')
    >>> print(opened_file_path)
    """
    logger = get_logger('core')
    logger.debug(f"Calling open")
    name = get_absolute_path(path)
    app.api.Open(name)
    return name

In [21]:
#| hide

app.open("test/공문양식.hwp")
app.close()

In [22]:
#| export
@patch
def get_hwnd(app: App):
    """
    Retrieves the window handle (HWND) of the active window in the Hancom Office Hwp program.

    This function accesses the active window in the Hancom Office Hwp program linked to the `App` instance
    and returns its window handle. The window handle can be used in scenarios where direct manipulation
    or interaction with the window at the OS level is required.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.

    Returns
    -------
    int
        The window handle (HWND) of the active Hancom Office Hwp window.

    Examples
    --------
    >>> app = App()
    >>> hwnd = get_hwnd(app)
    >>> print(hwnd)
    """
    logger = get_logger('core')
    logger.debug(f"Calling get_hwnd")
    return app.api.XHwpWindows.Active_XHwpWindow.WindowHandle

In [23]:
#| hide

app.get_hwnd()

5837054

In [24]:
#| export
@patch
def save(app: App, path=None):
    """
    Saves the current document in the Hancom Office Hwp program.

    If a path is provided, the document is saved to that location in the appropriate format based on the file extension.
    Supported formats include HWP, PDF, HWPML2X (HWPX), and PNG. If no path is provided, the document is saved
    in its current location.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    path : str, optional
        The file path where the document will be saved. If None, the document is saved in its current location.

    Returns
    -------
    str
        The file path where the document has been saved.

    Examples
    --------
    >>> app = App()
    >>> saved_file_path = save(app, 'path/to/document.hwp')
    >>> print(saved_file_path)
    """
    logger = get_logger('core')
    logger.debug(f"Calling save")
    if not path:
        app.api.Save()
        return app.get_filepath()
    
    name = get_absolute_path(path)
    extension = Path(name).suffix
    format_ = {
        ".hwp": "HWP",
        ".pdf": "PDF",
        ".hwpx": "HWPX",
        ".hml": "HWPML2X",
        ".png": "PNG",
        ".txt": "TEXT", 
        ".docx": "MSWORD", 
    }.get(extension)

    app.api.SaveAs(name, format_)
    return name

In [25]:
#| export
@patch
def save_block(app: App, path: Path):
    """
    Saves a block of content in the Hancom Office Hwp program and returns the file path.

    This function saves a specified block from the Hancom Office Hwp program to a file in a given format.
    The format is determined by the file extension. Supported formats include HWP, PDF, HWPML2X (HWPX), and PNG.
    It returns the file path if the save operation is successful or None if it fails.

    Parameters
    ----------
    app : App
        The application object where the action will be performed.
    path : Path
        The file path where the block will be saved.

    Returns
    -------
    Path or None
        The path to the saved file if the save operation is successful, or None if it fails.

    Examples
    --------
    >>> app = App()
    >>> saved_path = save_block(app, Path('path/to/save/block.hwp'))
    >>> print(saved_path)
    """
    logger = get_logger('core')
    logger.debug(f"Calling save_block")

    name = get_absolute_path(path)
    extension = Path(name).suffix
    format_ = {
        ".hwp": "HWP",
        ".pdf": "PDF",
        ".hwpx": "HWPML2X",
        ".png": "PNG",
    }.get(extension)

    action = app.actions.SaveBlockAction
    p = action.pset
    
    p.filename = name
    p.Format = format_
    action.run(p)
    return name if Path(name).exists() else None

In [26]:
action = app.actions.SaveBlockAction
p = action.pset
hasattr(p, "HSet")

False

In [27]:
#| hide

app.save_block("test2.hwp")

In [28]:
#| export
@patch
def close(app: App):
    """
    Closes the currently open document in the Hancom Office Hwp program.

    This function triggers the 'FileClose' command within the Hancom Office Hwp program to close the current document.
    It's useful for programmatically managing documents within the application.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.

    Examples
    --------
    >>> app = App()
    >>> # Open and manipulate the document
    >>> close(app)
    """
    logger = get_logger('core')
    logger.debug(f"Calling close")
    app.api.Run("FileClose")

In [29]:
#| hide

app.close()

In [30]:
#| export
@patch
def quit(app: App):
    """
    Terminates the Hancom Office Hwp program instance associated with the `App`.

    This function invokes the 'FileQuit' command within the Hancom Office Hwp program to close the application. 
    It's useful for programmatically controlling the lifecycle of the application.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.

    Examples
    --------
    >>> app = App()
    >>> # Perform actions with the app
    >>> quit(app)
    """
    logger = get_logger('core')
    logger.debug(f"Calling quit")
    app.api.Run("FileQuit")

In [31]:
#| export

@patch
def get_font_list(app:App):
    """
    Retrieves the list of fonts that used in the documnents from the current application.

    This method accesses the font list in the Hancom Office Hwp program linked to the `App` instance.
    It extracts fonts that used and return in list for further use or analysis.

    
    Parameters
    ----------
    app : App
        The application object from which the character shape settings will be retrieved.

    Returns
    -------
    List
        return font list that used in the hwp document.

    Examples
    --------
    >>> app = App()
    >>> font_list = app.get_font_list()
    >>> print(font_list)
    """
    logger = get_logger('core')
    logger.debug(f"Calling get_font_list")
    app.api.ScanFont()
    result = app.api.GetFontList()
    output = list(map(lambda txt: txt.split(",")[0],
        result.split("\x02")
        ))
    return output

In [32]:
#| export

@patch
def charshape(app:App, 
    facename=None,
    fonttype=None,
    size=None,
    ratio=None,
    spacing=None,
    offset=None,
    bold=None,
    italic=None,
    small_caps=None,
    emboss=None,
    engrave=None,
    superscript=None,
    subscript=None,
    underline_type=None,
    underline_shape=None,
    outline_type=None,
    shadow_type=None,
    text_color=None,
    shade_color=None,
    underline_color=None,
    shadow_color=None,
    shadow_offset_x=None,
    shadow_offset_y=None,
    strikeout_color=None,
    strikeout_type=None,
    strikeout_shape=None,
    diac_sym_mark=None,
    use_font_space=None,
    use_kerning=None,
    height=None,
    border_fill=None):
    """return null charshape
    """

    logger = get_logger('core')
    logger.debug(f"Calling charshape")

    charshape_pset = app.create_parameterset("CharShape")
    value_set = {
        "facename": facename,
        "fonttype": fonttype,
        "size": size,
        "ratio": ratio,
        "spacing": spacing,
        "offset": offset,
        "bold": bold,
        "italic": italic,
        "small_caps": small_caps,
        "emboss": emboss,
        "engrave": engrave,
        "superscript": superscript,
        "subscript": subscript,
        "underline_type": underline_type,
        "underline_shape": underline_shape,
        "outline_type": outline_type,
        "shadow_type": shadow_type,
        "text_color": text_color,
        "shade_color": shade_color,
        "underline_color": underline_color,
        "shadow_color": shadow_color,
        "shadow_offset_x": shadow_offset_x,
        "shadow_offset_y": shadow_offset_y,
        "strikeout_color": strikeout_color,
        "strikeout_type": strikeout_type,
        "strikeout_shape": strikeout_shape,
        "diac_sym_mark": diac_sym_mark,
        "use_font_space": use_font_space,
        "use_kerning": use_kerning,
        "height": height,
        "border_fil": border_fill}
    for key, value in value_set.items():
        if not key:
            continue
        setattr(charshape_pset, key, value)
        
    return charshape_pset

In [33]:
#| export
@patch
def get_charshape(app: App):
    """
    Retrieves the character shape settings from the current application.

    This method accesses the character shape settings in the Hancom Office Hwp program linked to the `App` instance.
    It extracts these settings and encapsulates them in a `CharShape` object for further use or analysis.

    Parameters
    ----------
    app : App
        The application object from which the character shape settings will be retrieved.

    Returns
    -------
    CharShape
        An object representing the character shape settings of the application.

    Examples
    --------
    >>> app = App()
    >>> char_shape = get_charshape(app)
    >>> print(char_shape)
    """
    logger = get_logger('core')
    logger.debug(f"Calling get_charshape")
    return app.actions.CharShape.pset

In [34]:
#| hide 

action = app.api.CreateAction("CharShape")
pset = app.api.HParameterSet.HCharShape
action.GetDefault(pset.HSet)
pset.Height

1000

In [35]:
#| hide

app.get_charshape()

{   'name': 'CharShape',
    'values': {   'Bold': 0,
                  'BorderFill': {   'CLSID': IID('{599CBB08-7780-4F3B-8ADA-7F2ECFB57181}'),
                                    'Clone': <bound method IDHwpParameterSet.Clone of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164560902624>>,
                                    'Count': 13,
                                    'CreateItemArray': <bound method IDHwpParameterSet.CreateItemArray of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164560902624>>,
                                    'CreateItemSet': <bound method IDHwpParameterSet.CreateItemSet of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164560902624>>,
                                    'GetIntersection': <bound method IDHwpParameterSet.GetIntersection of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164560902624>>,
                                 

In [36]:
#| export
@patch
def set_charshape(app: App, charshape: parametersets.CharShape=None, **kwargs):
    """
    Sets the character shape in the current paragraph of the Hancom Office Hwp application using the provided `CharShape`.

    If `charshape` is None, a default instance of `CharShape` is created. The function can also accept additional keyword 
    arguments (`kwargs`), which are assigned as properties of `charshape`. This modified `charshape` is then used to change 
    the current paragraph shape in the Hancom Office Hwp document.

    Parameters
    ----------
    app : App
        The `App` object. It is used to access the Hancom Office Hwp application.
    charshape : CharShape, optional
        The `CharShape` object to be used for setting character shapes in Hancom Office Hwp. The default is None.
    **kwargs
        Additional keyword arguments that are assigned as properties of `charshape`.

    Returns
    -------
    bool
        A boolean value indicating the success of the `set_charshape` operation.

    Examples
    --------
    >>> app = App()
    >>> char_shape = app.get_charshape()
    >>> success = set_charshape(app, charshape=char_shape, fontName='Arial', fontSize=10)
    >>> print(success)
    """
    logger = get_logger('core')
    logger.debug(f"Calling set_charshape")
    action = app.actions.CharShape
    pset = action.pset
    # charshape를 전달하면 반영해야 함
    if charshape:
        pset.update_from(charshape)

    for key, value in kwargs.items():
        setattr(pset, key, value)
    
    if action.run():
        return pset 
    return None

In [37]:
#| hide

charshape = app.get_charshape()
charshape
charshape.FaceNameHangul = "함초롬바탕"
charshape.super_script=1
charshape.sub_script = 1
app.set_charshape(charshape)

{   'name': 'CharShape',
    'values': {   'Bold': 0,
                  'BorderFill': {   'CLSID': IID('{599CBB08-7780-4F3B-8ADA-7F2ECFB57181}'),
                                    'Clone': <bound method IDHwpParameterSet.Clone of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164561688864>>,
                                    'Count': 13,
                                    'CreateItemArray': <bound method IDHwpParameterSet.CreateItemArray of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164561688864>>,
                                    'CreateItemSet': <bound method IDHwpParameterSet.CreateItemSet of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164561688864>>,
                                    'GetIntersection': <bound method IDHwpParameterSet.GetIntersection of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164561688864>>,
                                 

In [38]:
#| hide

app.get_charshape()

{   'name': 'CharShape',
    'values': {   'Bold': 0,
                  'BorderFill': {   'CLSID': IID('{599CBB08-7780-4F3B-8ADA-7F2ECFB57181}'),
                                    'Clone': <bound method IDHwpParameterSet.Clone of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164561689008>>,
                                    'Count': 13,
                                    'CreateItemArray': <bound method IDHwpParameterSet.CreateItemArray of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164561689008>>,
                                    'CreateItemSet': <bound method IDHwpParameterSet.CreateItemSet of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164561689008>>,
                                    'GetIntersection': <bound method IDHwpParameterSet.GetIntersection of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164561689008>>,
                                 

In [39]:
#| hide

charshape = app.get_charshape()

charshape.font = "바탕체"
charshape.super_script=0
app.set_charshape(charshape)

charshape.super_script == 0

True

In [40]:
#| export
@patch
def get_parashape(app: App):
    """
    Retrieves the paragraph shape settings from the current application.

    This method accesses the paragraph shape settings in the Hancom Office Hwp program linked to the `App` instance.
    It extracts these settings and encapsulates them in a `ParaShape` object for further use or analysis.

    Parameters
    ----------
    app : App
        The application object from which the paragraph shape settings will be retrieved.

    Returns
    -------
    ParaShape
        An object representing the paragraph shape settings of the application.

    Examples
    --------
    >>> app = App()
    >>> para_shape = get_parashape(app)
    >>> print(para_shape)
    """
    logger = get_logger('core')
    logger.debug(f"Calling get_parashape")
    return app.actions.ParagraphShape.pset

In [41]:
#| hide

dir(app.get_parashape())

['AlignType',
 'BorderConnect',
 'BorderFill',
 'BorderOffsetBottom',
 'BorderOffsetLeft',
 'BorderOffsetRight',
 'BorderOffsetTop',
 'BorderText',
 'BreakLatinWord',
 'BreakNonLatinWord',
 'Bullet',
 'Condense',
 'FontLineHeight',
 'HeadingType',
 'Indentation',
 'KeepLinesTogether',
 'KeepWithNext',
 'LeftMargin',
 'Level',
 'LineSpacing',
 'LineSpacingType',
 'LineWrap',
 'NextSpacing',
 'Numbering',
 'PagebreakBefore',
 'PrevSpacing',
 'REQUIRED_SETID',
 'RightMargin',
 'SnapToGrid',
 'TabDef',
 'TailType',
 'TextAlignment',
 'WidowOrphan',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakre

In [42]:
#| export

@patch
def set_parashape(app: App, parashape: ParaShape = None, **kwargs):
    """
    Sets the paragraph shape in the Hancom Office Hwp application using the provided `ParaShape`.

    If `parashape` is None, a default instance of `ParaShape` is created. The function can also accept additional keyword 
    arguments (`kwargs`), which are assigned as properties of `parashape`. This modified `parashape` is then used to change 
    the current paragraph shape in the Hancom Office Hwp document.

    Parameters
    ----------
    app : App
        The `App` object, used to access the Hancom Office Hwp application.
    parashape : ParaShape, optional
        The `ParaShape` object to be used for setting paragraph shapes in Hancom Office Hwp. Defaults to `None`.
    **kwargs
        Additional keyword arguments assigned as properties of `parashape`.
    if parashape:
        pset.update(parashape)

    bool
        A boolean value indicating the success of the `set_parashape` operation.

    Examples
    --------
    >>> app = App()
    >>> success = set_parashape(app, parashape=para_shape, align='Left', indent=10)
    >>> print(success)
    """
    logger = get_logger('core')
    logger.debug(f"Calling set_parashape")
    action = app.actions.ParagraphShape
    pset = action.pset
    if parashape:
        pset.update_from(parashape)

    for key, value in kwargs.items():
        setattr(pset, key, value)

    return action.run()

In [43]:
#| hide

app.set_parashape(line_spacing=100)

True

In [44]:
#| export
@patch
def insert_text(
    app: App,
    text: str,
    charshape: parametersets.CharShape = None, 
    **kwargs,
):
    """
    Inserts text into the Hancom Office Hwp document with specified character shape settings.

    This function inserts the given text into the document associated with the `App` instance. 
    It allows for optional character shape settings through a `CharShape` object. 
    Additional character shape attributes can be specified via keyword arguments.

    Parameters
    ----------
    app : App
        The `App` object associated with the Hancom Office Hwp program.
    text : str
        The text to be inserted into the document.
    charshape : CharShape, optional
        An optional `CharShape` object to specify the character shape settings for the inserted text. Defaults to `None`.
    **kwargs
        Additional character shape attributes to be set on the `charshape` object.

    Examples
    --------
    >>> app = App()
    >>> insert_text(app, "Hello World", fontName="Arial", fontSize=12)
    """

    
    logger = get_logger('core')
    logger.debug(f"Calling insert_text")
    
    if not charshape:
        charshape = app.get_charshape()
    for key, value in kwargs.items():
        setattr(charshape, key, value)
    app.set_charshape(charshape)
    
    insert_text = app.actions.InsertText
    p = insert_text.pset
    p.text = text
    
    insert_text.run(p)
    return

In [45]:
# hide

# test inside 

insert_text = app.actions.InsertText
p = insert_text.pset
p.text = "text"
insert_text.run()
p

{'name': 'InsertText', 'values': {'text': 'text'}}

In [46]:
#| hide
app.insert_text("테스트입니다.")
app.save("test.pdf")
app.save("test.png")
app.save("test.txt")

'c:/Users/freed/Documents/python_projects/007_hwpapi/nbs/02_api/test.txt'

In [47]:
#| hide

app.insert_text("안녕하세요", charshape)

In [48]:
#| hide

print(app.get_parashape())
app.set_parashape(left_margin=20000, align_type=0)

{   'name': 'ParaShape',
    'values': {   'AlignType': 0,
                  'BorderConnect': 0,
                  'BorderFill': {   'CLSID': IID('{599CBB08-7780-4F3B-8ADA-7F2ECFB57181}'),
                                    'Clone': <bound method IDHwpParameterSet.Clone of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164562054368>>,
                                    'Count': 13,
                                    'CreateItemArray': <bound method IDHwpParameterSet.CreateItemArray of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164562054368>>,
                                    'CreateItemSet': <bound method IDHwpParameterSet.CreateItemSet of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164562054368>>,
                                    'GetIntersection': <bound method IDHwpParameterSet.GetIntersection of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164562

True

In [49]:
#| hide

parashape = app.actions.ParagraphShape

In [50]:
#| hide

parashape.pset.right_margin

0

In [51]:
#| export


def _get_text(app):
    """스캔한 텍스트 텍스트 제너레이터"""
    flag, text = 2, ""
    while flag not in [0, 1, 101, 102]:
        flag, text = app.api.GetText()
        yield text

@patch
@contextmanager
def scan(
    app: App,
    option=const.MaskOption.All,
    selection=False,
    scan_spos=const.ScanStartPosition.Current,
    scan_epos=const.ScanEndPosition.Document,
    spara=None,
    spos=None,
    epara=None,
    epos=None,
    scan_direction=const.ScanDirection.Forward,
):

    
    logger = get_logger('core')
    logger.debug(f"Calling scan")

    range_ = scan_spos.value + scan_epos.value
    if selection:
        range_ = 0x00FF  # Limit the scanning to the block if selection is True

    range_ += scan_direction.value
    app.api.InitScan(
        option=option.value,
        Range=range_,
        spara=spara,
        spos=spos,
        epara=epara,
        epos=epos,
    )
    yield _get_text(app)
    app.api.ReleaseScan()

In [52]:
#| export


def move_to_line(app: App, text):
    """인자로 전달한 텍스트가 있는 줄의 시작지점으로 이동합니다."""
    logger = get_logger('core')
    logger.debug(f"Calling move_to_line")
    with app.scan(scan_spos=const.ScanStartPosition.Line) as scan:
        for line in scan:
            if text in line:
                return app.move.scan_pos()
    return False

In [53]:
#| hide 

move_to_line(app, "안녕하세요")

True

In [54]:
#| export

@patch
def setup_page(
    app: App,  # 앱 인스턴스
    top=20,  # 위쪽 여백 (밀리미터 단위)
    bottom=10,  # 아래쪽 여백 (밀리미터 단위)
    right=20,  # 오른쪽 여백 (밀리미터 단위)
    left=20,  # 왼쪽 여백 (밀리미터 단위)
    header=15,  # 머리글 길이 (밀리미터 단위)
    footer=5,  # 바닥글 길이 (밀리미터 단위)
    gutter=0,  # 제본 여백 (밀리미터 단위)
):  
    """
    Hancom Office Hwp 문서의 페이지 레이아웃을 설정합니다.

    이 함수는 페이지 여백, 머리글, 바닥글, 제본 여백의 크기를 설정합니다. 
    단위는 밀리미터이며, 애플리케이션 내부의 단위 시스템으로 변환됩니다.

    매개변수
    ----------
    app : App
        Hancom Office Hwp 프로그램과 연결된 `App` 인스턴스.
    top : int, optional
        위쪽 여백 크기 (밀리미터 단위). 기본값은 20mm.
    bottom : int, optional
        아래쪽 여백 크기 (밀리미터 단위). 기본값은 10mm.
    right : int, optional
        오른쪽 여백 크기 (밀리미터 단위). 기본값은 20mm.
    left : int, optional
        왼쪽 여백 크기 (밀리미터 단위). 기본값은 20mm.
    header : int, optional
        머리글 길이 (밀리미터 단위). 기본값은 15mm.
    footer : int, optional
        바닥글 길이 (밀리미터 단위). 기본값은 5mm.
    gutter : int, optional
        제본 여백 크기 (밀리미터 단위). 기본값은 0mm.

    반환값
    -------
    bool
        페이지 설정이 성공하면 True, 실패하면 False.

    사용 예시
    --------
    >>> app = App()
    >>> setup_page(app, top=25, bottom=15, right=20, left=20, header=10, footer=5, gutter=5)
    """


    
    logger = get_logger('core')
    logger.debug(f"Calling setup_page")
    
    action = app.actions.PageSetup
    p = action.pset

    # 각 여백 및 길이를 밀리미터 단위에서 한컴오피스 내부 단위로 변환하여 설정
    p.PageDef.TopMargin = app.api.MiliToHwpUnit(top)
    p.PageDef.HeaderLen = app.api.MiliToHwpUnit(header)
    p.PageDef.RightMargin = app.api.MiliToHwpUnit(right)
    p.PageDef.BottomMargin = app.api.MiliToHwpUnit(bottom)
    p.PageDef.FooterLen = app.api.MiliToHwpUnit(footer)
    p.PageDef.LeftMargin = app.api.MiliToHwpUnit(left)
    p.PageDef.GutterLen = app.api.MiliToHwpUnit(gutter)

    return action.run()  # 페이지 설정 실행

In [55]:
#| export


@patch
def insert_picture(
    app: App,
    fpath,
    width=None,
    height=None,
    size_option=const.SizeOption.RealSize,
    reverse=False,
    watermark=False,
    effect=const.Effect.RealPicture,
):
    """
    지정된 크기와 효과 옵션으로 문서에 이미지를 삽입합니다.

    매개변수
    ----------
    app : App
        Hancom Office Hwp 프로그램과 연결된 `App` 인스턴스.
    fpath : str
        삽입할 이미지 파일 경로.
    width : int, optional
        이미지의 너비. None이면 크기 옵션에 따라 결정됩니다. 기본값은 None.
    height : int, optional
        이미지의 높이. None이면 크기 옵션에 따라 결정됩니다. 기본값은 None.
    size_option : SizeOption, optional
        이미지의 크기 옵션으로 `SizeOption` Enum에 정의되어 있습니다. 기본값은 SizeOption.RealSize.
    reverse : bool, optional
        True이면 이미지를 반전합니다. 기본값은 False.
    watermark : bool, optional
        True이면 이미지를 워터마크로 처리합니다. 기본값은 False.
    effect : Effect, optional
        이미지의 시각적 효과로 `Effect` Enum에 정의되어 있습니다. 기본값은 Effect.RealPicture.

    반환값
    -------
    bool
        이미지가 성공적으로 삽입되었으면 True, 그렇지 않으면 False.

    사용 예시
    --------
    >>> app = App()
    >>> success = insert_picture(app, 'path/to/image.jpg', width=100, height=200, size_option=SizeOption.SpecificSize, effect=Effect.GrayScale)
    >>> print(success)
    """

    
    logger = get_logger('core')
    logger.debug(f"Calling insert_picture")

    path = Path(fpath)  # 이미지 파일 경로 처리
    size_option = size_option.value  # 크기 옵션 값 설정
    effect = effect.value  # 효과 옵션 값 설정

    return app.api.InsertPicture(
        path.absolute().as_posix(),  # 파일의 절대 경로
        Width=width,  # 이미지 너비
        Height=height,  # 이미지 높이
        sizeoption=size_option,  # 크기 옵션
        reverse=reverse,  # 반전 여부
        watermark=watermark,  # 워터마크 여부
        effect=effect,  # 시각적 효과
    )

In [56]:
#| export


@patch
def select_text(app: App, option=const.SelectionOption.Line):
    """
    지정된 옵션에 따라 문서에서 텍스트를 선택합니다.

    매개변수
    ----------
    app : App
        Hancom Office Hwp 프로그램과 연결된 `App` 인스턴스.
    option : SelectionOption, optional
        선택할 텍스트 단위. SelectionOption Enum에 정의된 옵션을 사용. 기본값은 SelectionOption.Line.

    반환값
    -------
    tuple
        시작 및 끝 이동 작업의 결과를 포함하는 튜플 (둘 다 boolean 값).

    사용 예시
    --------
    >>> app = App()
    >>> select_text(app, option=SelectionOption.Para)
    """

    logger = get_logger('core')
    logger.debug(f"Calling select_text with option={option}")

    # 문자열 입력 처리
    if isinstance(option, str):
        try:
            option = const.SelectionOption[option]  # 문자열을 Enum으로 변환
        except KeyError:
            raise ValueError(f"Invalid option string: {option}. Must be one of {[o.name for o in const.SelectionOption]}")

    # Enum 멤버라면 값 꺼내기
    begin_action_name, end_action_name = option.value
    begin_action = getattr(app.actions, begin_action_name)
    end_action = getattr(app.actions, end_action_name)

    return begin_action(), end_action()


In [57]:
#| hide


app.select_text

<bound method App.select_text of <Hwp App: >>

In [58]:
#| hide


app.set_charshape(facename="바탕", fontsize=20)

{   'name': 'CharShape',
    'values': {   'Bold': 0,
                  'BorderFill': {   'CLSID': IID('{599CBB08-7780-4F3B-8ADA-7F2ECFB57181}'),
                                    'Clone': <bound method IDHwpParameterSet.Clone of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164562056096>>,
                                    'Count': 13,
                                    'CreateItemArray': <bound method IDHwpParameterSet.CreateItemArray of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164562056096>>,
                                    'CreateItemSet': <bound method IDHwpParameterSet.CreateItemSet of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164562056096>>,
                                    'GetIntersection': <bound method IDHwpParameterSet.GetIntersection of <win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x2164562056096>>,
                                 

In [59]:
#| export
@patch
def get_selected_text(app: App):
    """
    Hancom Office Hwp 문서에서 현재 선택된 영역의 텍스트를 가져옵니다.

    이 함수는 문서에서 선택된 텍스트를 스캔하여 문자열로 반환합니다.
    현재 강조 표시되거나 선택된 텍스트를 처리하는 작업에 특히 유용합니다.

    매개변수
    ----------
    app : App
        Hancom Office Hwp 프로그램과 연결된 `App` 인스턴스.

    반환값
    -------
    str
        문서에서 현재 선택된 영역의 텍스트.

    사용 예시
    --------
    >>> app = App()
    >>> selected_text = get_selected_text(app)
    >>> print(selected_text)
    """
    logger = get_logger('core')
    logger.debug(f"Calling get_selected_text")

    with app.scan(selection=True) as scan:  # 선택된 영역 스캔
        text = "\n".join(scan)  # 선택된 텍스트를 줄 단위로 결합
    return text  # 결합된 텍스트 반환

In [60]:
#| hide


app.get_selected_text()

''

In [61]:
#| export 



# 리팩터링된 get_text 함수
@patch
def get_text(app: App, spos=const.ScanStartPosition.Line, epos=const.ScanEndPosition.Line):
    """
    지정된 시작 및 끝 위치에 따라 문서에서 텍스트를 가져옵니다.

    매개변수
    ----------
    app : App
        Hancom Office Hwp 프로그램과 연결된 `App` 인스턴스.
    spos : ScanStartPosition, optional
        텍스트 검색을 시작할 위치. 기본값은 ScanStartPosition.Line.
    epos : ScanEndPosition, optional
        텍스트 검색을 종료할 위치. 기본값은 ScanEndPosition.Line.

    반환값
    -------
    str
        지정된 시작 위치부터 끝 위치까지 가져온 텍스트.

    사용 예시
    --------
    >>> app = App()
    >>> text = get_text(app, spos=ScanStartPosition.Paragraph, epos=ScanEndPosition.Paragraph)
    >>> print(text)
    """
    logger = get_logger('core')
    logger.debug(f"Calling get_text")

    with app.scan(scan_spos=spos, scan_epos=epos) as txts:  # 지정된 위치 범위에서 텍스트 스캔
        text = "".join(txts)  # 스캔한 텍스트를 하나의 문자열로 결합
    return text  # 결합된 텍스트 반환

In [62]:
#| hide


app.get_text()

'text테스트입니다.안녕하세요\r\n'

In [ ]:
#| export 

@patch
def find_text(
    app: App,
    text="",  # 찾을 텍스트
    ignore_message=True,  # 메시지 무시 여부
    direction=0,  # 검색 방향
    match_case=None,  # 대소문자 구분
    all_word_forms=None,  # 모든 단어 형태 검색
    several_words=None,  # 여러 단어 검색
    use_wild_cards=None,  # 와일드카드 사용
    whole_word_only=None,  # 전체 단어만 검색
    replace_mode=None,  # 찾아 바꾸기 모드
    ignore_find_string=None,  # 찾을 문자열 무시
    ignore_replace_string=None,  # 바꿀 문자열 무시
    find_style="",  # 찾을 스타일
    replace_style="",  # 바꿀 스타일
    find_jaso=None,  # 자소로 검색
    find_regexp=None,  # 정규표현식으로 검색
    find_type=True,  # 마지막 검색(True), 새 검색(False)
    facename=None,
    text_color=None,
    bold = None,
    charshape: parametersets.CharShape = None, 

):
    """
    문서에서 다양한 옵션을 사용해 특정 텍스트를 검색합니다.

    매개변수
    ----------
    app : App
        Hancom Office Hwp 프로그램과 연결된 `App` 인스턴스.
    text : str
        검색할 텍스트 문자열.
    [기타 매개변수...]

    반환값
    -------
    bool
        텍스트를 찾았으면 True, 그렇지 않으면 False.

    사용 예시
    --------
    >>> app = App()
    >>> found = find_text(app, text="Hello", direction=Direction.Forward)
    >>> print(found)
    """

    
    logger = get_logger('core')
    logger.debug(f"Calling find_text")

    # 반복 검색 액션 생성
    action = app.actions.RepeatFind
    pset = action.pset
    # pset.FindCharShape.update_from(app.charshape())

    if charshape:
        pset.FindCharShape
        pset.FindCharShape.update_from(charshape)

    # 옵션 설정
    if text is not None:
        pset.FindString = text
    if ignore_message is not None:    
        pset.ignore_message = ignore_message
    if match_case is not None:    
        pset.mach_case = match_case
    if all_word_forms is not None:
        pset.all_word_forms = all_word_forms
    if direction is not None:
        pset.direction = direction
    if several_words is not None:
        pset.several_words = several_words
    if use_wild_cards is not None:
        pset.use_wild_cards = use_wild_cards
    if whole_word_only is not None:
        pset.whole_word_only = whole_word_only
    if replace_mode is not None:
        pset.replace_mode = replace_mode
    if ignore_find_string is not None:
        pset.IgnoreFindString = ignore_find_string
    if ignore_replace_string is not None:
        pset.IgnoreReplaceString = ignore_replace_string
    if find_style is not None:
        pset.FindStyle = find_style
    if replace_style is not None:
        pset.ReplaceStyle = replace_style
    if find_jaso is not None:
        pset.FindJaso = find_jaso
    if find_regexp is not None:
        pset.FindRegexp = find_regexp
    if find_type is not None:
        pset.FindType = find_type
    if facename is not None:
        pset.FindCharShape
        pset.FindCharShape.FaceNameHangul = facename
        from hwpapi.constants import korean_fonts, english_fonts
        fonts = set(korean_fonts + english_fonts)
        pset.FindCharShape.FontTypeHangul = 1 if facename not in fonts else 2
    if text_color is not None:
        pset.FindCharShape.TextColor = text_color
    if bold is not None:
        pset.FindCharShape.Bold = bold

    return action.run()

In [64]:
#| hide 

# set a test value
 
from hwpapi.parametersets import CharShape

app = App()
app.insert_text("안녕하세요", charshape=app.get_charshape(), facename="돋움", fonttype=1)
app.insert_text("안녕하세요", charshape=app.get_charshape(), facename="바탕", fonttype=1)


2025-12-08 13:22:26 - hwpapi.functions.get_hwp_objects - INFO - Found 1 running HWP objects
2025-12-08 13:22:26 - hwpapi.functions.dispatch - INFO - Successfully dispatched: <PyIDispatch at 0x000001F7F6D8A580 with obj at 0x000001F7F8122F78>
2025-12-08 13:22:26 - hwpapi.core - INFO - Engine initialized successfully with CLSID: {5E6A8276-CF1C-42B8-BCED-319548B02AF6}
2025-12-08 13:22:26 - hwpapi.core - INFO - Engine loaded successfully: {5E6A8276-CF1C-42B8-BCED-319548B02AF6}
2025-12-08 13:22:26 - hwpapi.core - INFO - Registering DLL: C:\Users\freed\Documents\python_projects\007_hwpapi\hwpapi\FilePathCheckerModuleExample.dll
2025-12-08 13:22:26 - hwpapi.core - INFO - App window visibility set to: True
2025-12-08 13:22:26 - hwpapi.core - INFO - App initialized successfully with all accessors


In [65]:
# hide

app.find_text("안녕하세요", charshape=CharShape(facename="돋움", fonttype=1))


AttributeError: 'NoneType' object has no attribute 'update_from'

In [ ]:
# hide

app.find_text("안녕하세요")

True

In [ ]:
#| hide

app.move.top_of_file()
# check it find text without style
count = 0
while app.find_text("안녕하세요"):
    count += 1

print("찾은 횟수", count)

찾은 횟수 3


In [ ]:
#| hide



# run with action

act = app.actions.RepeatFind
pset = act.pset
pset.FindCharShape.FaceNameHangul = "돋움"
pset.FindCharShape.FontTypeHangul = 1
pset.FindString = "안녕하세요"
pset.apply()
count = 0
while act.run():
    count += 1
print('반복횟수:', count)

C:\Users\freed\Documents\python_projects\007_hwpapi\hwpapi\parametersets.py:3242: SyntaxWarning: invalid escape sequence '\_'
  """


KeyError: "Cannot set 'FindCharShape': must be real number, not dict"

In [ ]:
action = app.api.CreateAction("CharShape")
action.GetDefault(app.api.HParameterSet.HFindReplace.FindCharShape.HSet)

In [ ]:
app = App()

hwp = app.api

action = hwp.CreateAction("RepeatFind")
action.GetDefault(hwp.HParameterSet.HFindReplace.HSet)
hwp.HParameterSet.HFindReplace.FindCharShape.FaceNameHangul = "돋움"
hwp.HParameterSet.HFindReplace.FindCharShape.FaceNameLatin = "돋움"
hwp.HParameterSet.HFindReplace.FindCharShape.FaceNameHanja = "돋움"
hwp.HParameterSet.HFindReplace.FindCharShape.FaceNameJapanese = "돋움"
hwp.HParameterSet.HFindReplace.FindCharShape.FaceNameOther = "돋움"
hwp.HParameterSet.HFindReplace.FindCharShape.FaceNameSymbol = "돋움"
hwp.HParameterSet.HFindReplace.FindCharShape.FaceNameUser = "돋움"
hwp.HParameterSet.HFindReplace.FindCharShape.FontTypeHangul = 1
hwp.HParameterSet.HFindReplace.FindCharShape.FontTypeLatin = 1
hwp.HParameterSet.HFindReplace.FindCharShape.FontTypeHanja = 1
hwp.HParameterSet.HFindReplace.FindCharShape.FontTypeJapanese = 1
hwp.HParameterSet.HFindReplace.FindCharShape.FontTypeOther = 1
hwp.HParameterSet.HFindReplace.FindCharShape.FontTypeSymbol = 1
hwp.HParameterSet.HFindReplace.FindCharShape.FontTypeUser = 1

action.Execute(hwp.HParameterSet.HFindReplace.HSet)

2025-10-10 12:20:17 - hwpapi.functions.get_hwp_objects - INFO - Found 1 running HWP objects
2025-10-10 12:20:17 - hwpapi.functions.dispatch - INFO - Successfully dispatched: <PyIDispatch at 0x000001661B237D70 with obj at 0x000001661B28E4F8>
2025-10-10 12:20:17 - hwpapi.core - INFO - Engine initialized successfully with CLSID: {5E6A8276-CF1C-42B8-BCED-319548B02AF6}
2025-10-10 12:20:17 - hwpapi.core - INFO - Engine loaded successfully: {5E6A8276-CF1C-42B8-BCED-319548B02AF6}
2025-10-10 12:20:17 - hwpapi.core - INFO - Registering DLL: C:\Users\freed\Documents\python_projects\007_hwpapi\hwpapi\FilePathCheckerModuleExample.dll
2025-10-10 12:20:17 - hwpapi.core - INFO - App window visibility set to: True
2025-10-10 12:20:17 - hwpapi.core - INFO - App initialized successfully with all accessors


True

In [ ]:
action = hwp.CreateAction("RepeatFind")
pset = action.CreateSet()
action.GetDefault(pset)
pset.SetItem("FindString", "안녕하세요")
print(pset.Item("FindCharShape"))
# print(pset.ItemExist("FindCharShape"))
charshaped = pset.CreateItemSet("FindCharShape", "CharShape")
parashaped = pset.CreateItemSet("FindParaShape", "ParaShape")
print(pset.Item("FindCharShape"))
# charshaped.SetItem("Bold", True)
print(charshaped.SetID)
action.Execute(pset)

<win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x1538068699600>
<win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x1538049846848>
CharShape


<win32com.gen_py.HwpObject 1.0 Type Library.IDHwpParameterSet instance at 0x1538068705552>

In [ ]:
action = hwp.CreateAction("RepeatFind")
action.GetDefault(hwp.HParameterSet.HFindReplace.HSet)
hwp.HParameterSet.HFindReplace.FindString = "안녕하세요"
action.Execute(hwp.HParameterSet.HFindReplace.HSet)

In [ ]:
act.app.actions.FindDlg()

In [ ]:
#| hide

# run with action

act.app.actions.FindDlg()
act = app.actions.RepeatFind
pset = act.pset
pset.FindType = 1
pset.FindCharShape
pset.FindString = "안녕하세요"

count = 0
while act.run():
    count += 1
    print(count)

In [ ]:
pset.FindCharShape

In [ ]:
dir()

In [ ]:
#| hide

# check it find text with certain charshape
# 
count = 0 
while app.find_text("안녕하세요", charshape=CharShape(facename="돋움", fonttype=1)):
    count += 1
print("찾은 횟수", count)

In [ ]:
#| hide

# check it find text with certain charshape
# 
count = 0 
while app.find_text("안녕하세요", facename="돋움",):
    count += 1
print("찾은 횟수", count)


In [ ]:
app.insert_text("안녕하세요")

app.get_charshape()

In [ ]:
action = app.actions.RepeatFind
pset = app.api.HParameterSet.HFindReplace
pset.FindCharShape.FaceNameHangul = "돋움"
pset.FindCharShape.FaceNameLatin = "돋움"
pset.FindCharShape.FaceNameJapanese = "돋움"
pset.FindCharShape.FaceNameHanja = "돋움"
pset.FindCharShape.FaceNameOther = "돋움"
pset.FindCharShape.FaceNameSymbol = "돋움"
pset.FindCharShape.FaceNameUser = "돋움"
pset.FindCharShape.FontTypeHangul = 1
pset.FindCharShape.FontTypeLatin = 1
pset.FindCharShape.FontTypeJapanese = 1
pset.FindCharShape.FontTypeHanja = 1
pset.FindCharShape.FontTypeOther = 1
pset.FindCharShape.FontTypeSymbol = 1
pset.FindCharShape.FontTypeUser = 1
pset.FindString="안녕하세요"

action.run(pset)


In [ ]:
dir(pset.FindCharShape)

In [ ]:
#| hide



app.find_text("안녕하세요")
app.find_text("안녕")
app.find_text("하세요")



In [ ]:
action = app.actions.RepeatFind
pset = action.pset
pset.FindCharShape

In [ ]:
action = app.actions.RepeatFind
pset = action.pset
pset.FindCharShape.update_from(app.get_charshape())
pset.FindCharShape.FaceNameHangul = "돋움"
pset.FindCharShape.FontTypeHangul = 1
pset.FindCharShape.Size = None
print(pset.FindCharShape)
pset.FindString
action.run()

In [ ]:
app.find_text("안녕하세요")

In [ ]:
action = app.actions.RepeatFind
pset = action.pset
pset.FindString = "안녕하세요"
pset.apply()
for key in dir(app.api.HParameterSet.HFindReplace):
    print(key, getattr(app.api.HParameterSet.HFindReplace, key))

for key in dir(app.api.HParameterSet.HFindReplace.FindCharShape):
    print(key, getattr(app.api.HParameterSet.HFindReplace.FindCharShape, key))

action.run()

In [ ]:
app.find_text("안")
app.find_text("안", charshape=CharShape(facename="가는안상수체", size=21))


In [ ]:
app.get_charshape()

In [ ]:
#| export
@patch
def replace_all(
    app: App,
    old_text="",
    new_text="",
    old_charshape=None,  # 기존 문자 모양
    new_charshape=None,  # 새로운 문자 모양
    ignore_message=True,  # 메시지 무시 여부
    direction=None,  # 검색 방향
    match_case=None,  # 대소문자 구분
    all_word_forms=None,  # 모든 단어 형태
    several_words=None,  # 여러 단어 검색
    use_wild_cards=None,  # 와일드카드 사용
    whole_word_only=None,  # 전체 단어만 검색
    auto_spell=None,  # 자동 철자 교정
    replace_mode=None,  # 찾아 바꾸기 모드
    ignore_find_string=None,  # 찾을 문자열 무시
    ignore_replace_string=None,  # 바꿀 문자열 무시
    find_regexp=None,  # 정규표현식으로 검색
    find_style=None,  # 찾을 스타일
    replace_style=None,  # 바꿀 스타일
    find_jaso=None,  # 자소로 검색
    find_reg_exp=None,  # 정규표현식으로 검색
    find_type=None,  # 마지막 검색 사용(True), 새 검색(False)
):
    """
    문서에서 특정 텍스트를 새 텍스트로 모두 교체합니다.

    매개변수
    ----------
    app : App
        Hancom Office Hwp 프로그램과 연결된 `App` 인스턴스.
    old_text : str
        교체할 텍스트 문자열.
    new_text : str
        대체할 새 텍스트 문자열.
    [기타 매개변수...]

    반환값
    -------
    bool
        교체 작업이 성공하면 True, 실패하면 False.

    사용 예시
    --------
    >>> app = App()
    >>> success = replace_all(app, old_text="Hello", new_text="Hi", direction=Direction.All)
    >>> print(success)
    """


    logger = get_logger('core')
    logger.debug(f"Calling replace_all")

    # 반복 검색 액션 생성
    action = app.actions.AllReplace
    pset = action.pset

    # 옵션 설정
    if old_text is not None:
        pset.FindString = old_text
    if new_text is not None:
        pset.ReplaceString = new_text
    if ignore_message is not None:    
        pset.ignore_message = ignore_message
    if match_case is not None:    
        pset.mach_case = match_case
    if all_word_forms is not None:
        pset.all_word_forms = all_word_forms
    if direction is not None:
        pset.direction = direction
    if several_words is not None:
        pset.several_words = several_words
    if use_wild_cards is not None:
        pset.use_wild_cards = use_wild_cards
    if whole_word_only is not None:
        pset.whole_word_only = whole_word_only
    if replace_mode is not None:
        pset.replace_mode = replace_mode
    if ignore_find_string is not None:
        pset.IgnoreFindString = ignore_find_string
    if ignore_replace_string is not None:
        pset.IgnoreReplaceString = ignore_replace_string
    if find_style is not None:
        pset.FindStyle = find_style
    if replace_style is not None:
        pset.ReplaceStyle = replace_style
    if find_jaso is not None:
        pset.FindJaso = find_jaso
    if find_regexp is not None:
        pset.FindRegexp = find_regexp
    if find_type is not None:
        pset.FindType = find_type
    if auto_spell is not None:
        pset.auto_spell = auto_spell

    # 문자 모양 설정
    if old_charshape:
        pset.FindCharShape.update_from(old_charshape)
    if new_charshape:
        pset.replace_charshape.update_from(new_charshape)
    return action.run()

In [ ]:
#| hide


app.insert_text("t")

In [ ]:
action = app.actions.AllReplace
pset = action.pset

pset

In [ ]:
#| hide

app.replace_all("t", "txt", new_charshape=app.charshape(facename="가는안상수체", size=21))

In [ ]:
#| export
@patch
def insert_file(
    app: App,
    fpath,
    keep_charshape=False,
    keep_parashape=False,
    keep_section=False,
    keep_style=False,
):
    """
    Inserts the contents of a specified file into the current document.

    This function inserts the contents of another file into the current document at the cursor's position. 
    It provides options to retain various formatting attributes of the inserted content.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    fpath : str
        The file path of the document to be inserted.
    keep_charshape : bool, optional
        If True, retains the original character shapes from the inserted file. Defaults to False.
    keep_parashape : bool, optional
        If True, retains the original paragraph shapes from the inserted file. Defaults to False.
    keep_section : bool, optional
        If True, retains the original section formatting from the inserted file. Defaults to False.
    keep_style : bool, optional
        If True, retains the original styles from the inserted file. Defaults to False.

    Returns
    -------
    bool
        True if the file was inserted successfully, False otherwise.

    Examples
    --------
    >>> app = App()
    >>> success = insert_file(app, 'path/to/file.hwp', keep_style=True)
    >>> print(success)
    """


    
    logger = get_logger('core')
    logger.debug(f"Calling insert_file")

    
    action = app.actions.InsertFile
    p = action.pset
    p.filename = Path(fpath).absolute().as_posix()
    p.KeepCharshape = keep_charshape
    p.KeepParashape = keep_parashape
    p.KeepSection = keep_section
    p.KeepStyle = keep_style

    return action.run()

In [ ]:
#| export






@patch
def set_cell_border(
    app: App,
    top=None,
    right=None,
    left=None,
    bottom=None,
    horizontal=None,
    vertical=None,
    top_width=const.Thickness.NULL,
    right_width=const.Thickness.NULL,
    left_width=const.Thickness.NULL,
    bottom_width=const.Thickness.NULL,
    horizontal_width=const.Thickness.NULL,
    vertical_width=const.Thickness.NULL,
    top_color=None,
    bottom_color=None,
    left_color=None,
    right_color=None,
    horizontal_color=None,
    vertical_color=None,
):
    """
    Sets the border properties for cells in a table within a Hancom Office Hwp document.

    This function customizes the border types, widths, and colors for different sides of the cells. 
    It allows for detailed customization of cell appearance in tables.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    top, right, left, bottom, horizontal, vertical : int, optional
        Types of borders for the respective sides and internal lines of the cell (e.g., solid, dashed).
    top_width, right_width, left_width, bottom_width, horizontal_width, vertical_width : Thickness, optional
        Widths of the borders corresponding to the sides and internal lines of the cell. Specified using the `Thickness` Enum.
    top_color, bottom_color, left_color, right_color, horizontal_color, vertical_color : str, optional
        Colors for the respective borders in a string format (e.g., "#RRGGBB").

    Returns
    -------
    bool
        True if the border settings were successfully applied, False otherwise.

    Examples
    --------
    >>> app = App()
    >>> success = set_cell_border(app, top=1, bottom=1, top_width=Thickness._0_25_mm, bottom_width=Thickness._0_25_mm, top_color="#FF0000", bottom_color="#00FF00")
    >>> print(success)

    Notes
    -----
    The function relies on the `app.actions.CellFill` action to set the border properties. The `Thickness` Enum provides predefined thickness levels for the borders. The color parameters should be provided in hex format.
    """
    attrs = {
        "BorderTypeTop": top,
        "BorderTypeRight": right,
        "BorderTypeLeft": left,
        "BorderTypeBottom": bottom,
        "TypeHorz": horizontal,
        "TypeVert": vertical,
        "BorderWidthTop": top_width.value,
        "BorderWidthRight": right_width.value,
        "BorderWidthLeft": left_width.value,
        "BorderWidthBottom": bottom_width.value,
        "WidthHorz": horizontal_width.value,
        "WidthVert": vertical_width.value,
        "BorderColorTop": app.api.RGBColor(get_rgb_tuple(top_color))
        if top_color
        else None,
        "BorderColorRight": app.api.RGBColor(get_rgb_tuple(right_color))
        if right_color
        else None,
        "BorderColorLeft": app.api.RGBColor(get_rgb_tuple(left_color))
        if left_color
        else None,
        "BorderColorBottom": app.api.RGBColor(get_rgb_tuple(bottom_color))
        if bottom_color
        else None,
        "ColorHorz": app.api.RGBColor(get_rgb_tuple(horizontal_color))
        if horizontal_color
        else None,
        "ColorVert": app.api.RGBColor(get_rgb_tuple(vertical_color))
        if vertical_color
        else None,
    }

    
    logger = get_logger('core')
    logger.debug(f"Calling set_cell_border")

    action = app.actions.CellFill
    p = action.pset

    for key, value in attrs.items():
        if value is None:
            continue
        setattr(p, key, value)

    return action.run()

In [ ]:
#| export
@patch
def set_cell_color(
    app: App, bg_color=None, hatch_color="#000000", hatch_style=6, alpha=None
):
    """
    Sets the background color and hatch style for cells in a Hancom Office Hwp document.

    This function allows customization of the cell background, including color and hatching pattern,
    providing options to set transparency and hatch color.

    Parameters
    ----------
    app : App
        The `App` instance associated with the Hancom Office Hwp program.
    bg_color : str, optional
        Hexadecimal color code for the cell background (e.g., "#RRGGBB"). If not specified, the background is not changed.
    hatch_color : str, optional
        Hexadecimal color code for the hatching. Default is black ("#000000").
    hatch_style : int, optional
        Style of the hatching pattern. Default is 6.
    alpha : int, optional
        Alpha value for the background color's transparency (0-255). If not specified, transparency is not changed.

    Returns
    -------
    bool
        True if the cell color settings were applied successfully, False otherwise.

    Examples
    --------
    >>> app = App()
    >>> success = set_cell_color(app, bg_color="#FF0000", hatch_color="#0000FF", hatch_style=5, alpha=128)
    >>> print(success)

    Notes
    -----
    The function uses the `app.actions.CellBorderFill` action for setting the cell properties.
    Colors are specified in hexadecimal format. The alpha parameter controls the transparency of the background color.
    """


    logger = get_logger('core')
    logger.debug(f"Calling set_cell_color")

    fill_type = windows_brush = None
    if bg_color:
        fill_type = 1
        windows_brush = 1

    attrs = {
        "type": fill_type,
        "WindowsBrush": windows_brush,
        "WinBrushFaceColor": app.api.RGBColor(*get_rgb_tuple(bg_color)) if bg_color else None,
        "WinBrushAlpha": alpha,
        "WinBrushHatchColor": app.api.RGBColor(*get_rgb_tuple(hatch_color)) if hatch_color else None,
        "WinBrushFaceStyle": hatch_style,
    }

    action = app.actions.CellBorderFill
    p = action.pset

    for key, value in attrs.items():
        if value is not None:
            setattr(p.FillAttr, key, value)

    return action.run()

In [ ]:
#| hide


app = App()

In [ ]:
#| hide


app.set_charshape(app.charshape(facename="문화바탕", fonttype=2))

In [ ]:
#| hide


action = app.actions.CharShape

In [ ]:
#| hide


app.get_charshape()

In [ ]:
#| hide


app.api.HwpLineWidth("1.0mm")

In [ ]:
#| hide

app.set_cell_border(bottom=3)

In [ ]:
#| hide

get_rgb_tuple("#123456")

In [ ]:
#| hide

app.set_cell_color(bg_color="#123456")

In [ ]:
#| hide

# app.api.CurSelectedCtrl.CtrlID

In [ ]:
#| hide

# 동작은 되나 너무 느리다. 
# 빠르게 할 수 있는 방법을 찾아보자. 
# 양식을 설정할 수 있는 방법을 생각해 보자.
# 어떻게 해야 위아래로 값이 같은 셀을 병합할 수 있을지 생각해 보자.
# block_input을 해야 그나마 쓸 수 있을 듯

def insert_dataframe(app, df):
    logger = get_logger('core')
    logger.debug(f"Calling insert_dataframe")
    action = app.actions.TableCreate
    p = action.pset
    p.WidthType = 1    # fit size
    p.Rows = df.shape[0]+1  # add header
    p.Cols = df.shape[1]
    action.run()

    for col in df.columns:
        app.insert_text(col)
        app.move("RightOfCell")
    for i, row in df.iterrows():
        for item in row.values:
            app.insert_text(item)
            app.move("RightOfCell")
    return app.api.ParentCtrl

In [ ]:
#| hide


## 더 빠르고 안정적일 듯
# 초단위로 걸려서 block_input을 해야 할 듯 
# value 중에서 tab이 있으면 replace는 해야 할 듯

def convert_to_text(value):
    logger = get_logger('core')
    logger.debug(f"Calling convert_to_text")
    value = str(value).replace("\t", "    ")
    return value

def insert_dataframe(app:App, df):
    start_pos = app.api.GetPos()
    text = ""
    text += "\t".join(map(convert_to_text, df.columns))
    for i, row in df.iterrows():
        text += "\r\n"
        text += "\t".join(map(convert_to_text, row.values))
    app.insert_text(text)
    end_pos = app.api.GetPos()

    app.api.SelectText(start_pos[1], start_pos[2], end_pos[1], end_pos[2])

    return app.actions.TableStringToTable.run()

In [ ]:
#| hide

app = App()
app.page.inner_height

In [ ]:
#| hide

import pandas as pd
import numpy as np



In [ ]:
#| hide

df = pd.DataFrame(np.random.random(100).reshape(10, 10))
dir(app.api)


#| hide

## 

cell을 순환하면서 모양을 잡을 수 있을 것으로 예상된다.
6개 값을 받아서 정렬하면 되지 않을까 싶다.


In [ ]:
#| hide

insert_dataframe(app, df)
app.api.FindCtrl()
app.actions.ShapeObjTextBoxEdit.run()
app.set_cell_border(False, False)
for _ in range(10):
    app.actions.TableCellBlockRow.run()
    app.set_cell_color(bg_color=(253, 123, 253), hatch_style=6)
    app.set_cell_border(False, False, False, False)

    app.actions.TableCellBlock.run()
    app.move("DownOfCell")

In [ ]:
pset = app.api.CellShape
pset.Item("Width")

In [ ]:
app.api.ScanFont()
app.api.GetFontList()
app.api.ViewProperties

In [ ]:
dir(app.api)

In [ ]:
import re


TOKENS = [
    ('HEADER1', r'(^|\n)# (.*?)(\n|$)'),
    ('HEADER2', r'(^|\n)## (.*?)(\n|$)'),
    ('HEADER3', r'(^|\n)### (.*?)(\n|$)'),
    ('HEADER4', r'(^|\n)#### (.*?)(\n|$)'),
    ('HEADER5', r'(^|\n)##### (.*?)(\n|$)'),
    ('HEADER6', r'(^|\n)###### (.*?)(\n|$)'),
    ('BOLD', r'\*\*(.*?)\*\*'),
    ('ITALIC', r'\*(.*?)\*'),
    ('LIST_ITEM', r'(^|\n)( *)(-|\+|\*) (.*?)(\n|$)'),
    ('ORDERED_LIST_ITEM', r'(^|\n)( *)(\d+)\. (.*?)(\n|$)'),
    ('TABLE_ROW', r'(^|\n)\|.*?\|(\n|$)'),
    ('TABLE_SEPARATOR', r'(^|\n)\|[-\s|]*?\|(\n|$)'),
    ('NEWLINE', r'\n'),
    ('TEXT', r'[^#\*\-\n\|]+')
]


import re

def tokenize(text):
    logger = get_logger('core')
    logger.debug(f"Calling tokenize")
    tokens = []
    while text:
        for token_type, pattern in TOKENS:
            match = re.match(pattern, text)
            if match:
                if token_type == 'LIST_ITEM' or token_type == 'ORDERED_LIST_ITEM':
                    indent = len(match.group(2))
                    tokens.append(('LIST_ITEM', indent, match.group(4).strip()))
                else:
                    tokens.append((token_type, match.group().strip()))
                text = text[match.end():]
                break
        else:
            # 매칭되는 패턴이 없는 경우 텍스트에서 첫 글자를 잘라냄
            text = text[1:]
    return tokens

markdown_text = """
# Hello World

This is a **bold** text and this is *italic* text.

- Item 1
  - Subitem 1.1
  - Subitem 1.2
    - Subsubitem 1.2.1
- Item 2
"""

tokens = tokenize(markdown_text)
for token in tokens:
    print(token)

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()